In [173]:
import requests
import wikipedia as wiki
import spacy
nlp = spacy.load("en_core_web_sm") 
from bs4 import BeautifulSoup as soup
import pandas as pd
from IPython.display import Image
from IPython.core.display import HTML 
from datetime import date

In [174]:
def get_wikiname(name):
    wikiname = wiki.search(name)[:1]
    return wikiname

In [175]:
def get_wikiurl(wikiname):
    url = wiki.page(wikiname).url
    return url

In [176]:
def get_wikipicture(wikiname):
    image = wiki.page(wikiname).images[0]
    return image

In [177]:
def get_friends(wikiname):
    content = wiki.page(wikiname).content
    doc = nlp(content)
    ents_list = []
    for ent in doc.ents:
        if ent.label_=="PERSON" and ent.text not in ents_list and 3 > len(ent.text.split()) > 1:
            ents_list.append(ent.text)
    return ents_list

In [178]:
name="obama"
wikiname = get_wikiname(name)
level_1 = get_friends(wikiname)

In [179]:
rows = []
for name in level_1:
    name = name.replace(" ", "_")
    try:
        link_data = f'http://dbpedia.org/data/{name}.json'
        data = requests.get(link_data).json()
        link_person = f'http://dbpedia.org/resource/{name}'
        person = data[link_person]
    except:
        pass
    try:
        birthplace = person['http://dbpedia.org/property/birthPlace'][0]['value']
    except:
        birthplace = 'NaN'
    try:
        bday = person['http://dbpedia.org/ontology/birthDate'][0]['value']
    except:
        bday = 'NaN'
    try:
        url = get_wikiurl(name)
    except:
        url = 'NaN'
    #picture = get_wikipicture(name)
    #Image(picture, width = 100, height = 50)             
    rows.append([name.replace("_", " "), birthplace, bday, url])  
df = pd.DataFrame(rows, columns=["Name", "Birthplace", "Birthday", "Url"])

C:\Anaconda3\envs\wiki\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Anaconda3\envs\wiki\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [180]:
def calculate_age(dtob):
    today = date.today()
    return today.year - dtob.year - ((today.month, today.day) < (dtob.month, dtob.day))
age_list = []
for i in range(len(df)):
    try:
        year = int(df.iloc[i]['Birthday'][0:4])
        month = int(df.iloc[i]['Birthday'][5:7])
        day = int(df.iloc[i]['Birthday'][8:10])
        age_list.append(calculate_age(date(year,month,day)))
    except:
        age_list.append("NaN")
        continue
df['Age'] = age_list

In [182]:
df.head(10)

,Name,Birthplace,Birthday,Url,Age
0,Hillary Clinton,"Chicago, Illinois, U.S.",1947-10-26,https://en.wikipedia.org/wiki/Hillary_Clinton,73
1,John McCain,"Coco Solo, Panama Canal Zone, U.S.",1936-08-29,NaN,84
2,Joe Biden,"Scranton, Pennsylvania, U.S.",1942-11-20,NaN,78
3,Muammar Gaddafi,http://dbpedia.org/resource/Qasr_Abu_Hadi,NaN,https://en.wikipedia.org/wiki/Muammar_Gaddafi,NaN
4,Mitt Romney,"Detroit, Michigan, U.S.",1947-03-12,https://en.wikipedia.org/wiki/Mitt_Romney,74
5,Sonia Sotomayor,"New York City, U.S.",1954-06-25,https://en.wikipedia.org/wiki/Sonia_Sotomayor,66
6,Elena Kagan,"New York City, U.S.",1960-04-28,NaN,60
7,Merrick Garland,"Chicago, Illinois, U.S.",1952-11-13,NaN,68
8,Mitch McConnell,"Sheffield, Alabama, U.S.",1942-02-20,https://en.wikipedia.org/wiki/Mitch_McConnell,79
9,Ann Dunham,"Wichita, Kansas, U.S.",1942-11-29,NaN,78
